# Interactive Fitting of the Hit Rate Curves

This interactive notebook allows the user to manually and interactively experiment with parameters of the hit rate model.

## Prepare Inputs for Examples
### Load Libraries

In [1]:
# --- Import necessary libraries ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import ipywidgets as widgets

from model import HitRateModel

### Load Experimental Data

In [2]:
# Load experimental hit rate data with confidence intervals
df = pd.read_csv("data/hitrates_conf_intervals.csv")

# Group the data by target for easy access
data = {target: subset for target, subset in df.groupby("target")}
n_targets = len(data)

### Setup a Class for the Interactive Plot

In [3]:
class InteractivePlot:
    def __init__(self, data, pprop_lims=(0.5, 7.5)):
        self.data = data # Dictionary of dataframes keyed by target.
        self.pprop_lims = pprop_lims # Range of pProp values to consider.
        self.pprop = np.linspace(*pprop_lims, 50) # pProp values for plotting

    def interactive_plot(self, target, artifact_mean, artifact_std, p_art_freq, rho, exp_mean, exp_std):
        # Prepare target data
        df_exp = self.data[target][["pprop", "hit_rate_mean", "definition"]].copy()
        df_exp["Experimental"] = True
        definitions = df_exp["definition"].unique()

        # Initialize model
        model_params = {
            "artifact_mean": artifact_mean,
            "artifact_std": artifact_std,
            "artifact_freq": 10 ** (-p_art_freq),
            "rho": -rho,
            "exp_mean": exp_mean,
            "exp_std": exp_std}
        hrm = HitRateModel(model_params)

        # Generate predictions for each definition
        df_list = [df_exp]
        for def_val in definitions:
            predicted_hr = hrm.get_pprop_hr(self.pprop, np.full_like(self.pprop, def_val))
            # A new DataFrame is generated for each definition
            df_pred = pd.DataFrame({
                "pprop": self.pprop,
                "hit_rate_mean": predicted_hr,
                "definition": def_val,
                "Experimental": False
            })
            df_list.append(df_pred)

        # Concatenate and scale hit rate
        df_all = pd.concat(df_list)
        df_all["hit_rate_mean"] *= 100
        df_all["definition"] = df_all["definition"].apply(lambda x: f"{x:.2f}")

        # Plot
        fig, ax = plt.subplots(figsize=(6, 4))
        sns.set_theme()
        sns.lineplot(
            data=df_all,
            x="pprop",
            y="hit_rate_mean",
            hue="definition",
            style="Experimental",
            palette="Set1",
            ax=ax
        )
        ax.set_xlim(*self.pprop_lims)
        ax.set_ylim(0, 100)
        ax.set_xlabel("pProp (–log(fractional rank))")
        ax.set_ylabel("Hit Rate (%)")
        ax.set_title(f"Target: {target}")

        plt.tight_layout()
        plt.show()

## Run the Interactive Plot

In [4]:
plot = InteractivePlot(data=data, pprop_lims=(0.5,7.5))

# Create interactive widget
interact(
    plot.interactive_plot, 
    target=data.keys(),
    artifact_mean=(-6, 0, 0.1), 
    artifact_std=(0.1, 1, 0.1), 
    p_art_freq=(0, 7, 0.1), 
    rho=(0, 0.95, 0.05), 
    exp_mean=(-4, 4, 0.1),
    exp_std=(0.5, 3, 0.1))

interactive(children=(Dropdown(description='target', options=('ampc', 'd4', 'sigma2'), value='ampc'), FloatSli…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>